In [ ]:
import pandas as pd

file=open('C:/Users/USER/Desktop/플젝/춘향전-상황1.txt','r')
all=file.read()
file.close()
lines=all.split('\n')
chat=lines
chat=pd.DataFrame(chat)
chat=chat.rename(columns={0:'chat'})


chat['chat']=chat['chat'].str.replace(';',':')
chat['chat']=chat['chat'].str.replace(':','★')
chat['chat']=chat['chat'].str.replace('[', '')

for a,b in enumerate(chat['chat']):
    if '★' in chat['chat'][a]:
        pass
    else:
        chat=chat.drop([a])
        
chat=chat.reset_index()

In [ ]:
name=[]
chat_kor=[]
for a,b in enumerate(chat['chat']):
    name.append(chat['chat'][a].split('★',2)[0])
for a,b in enumerate(chat['chat']):
    chat_kor.append(chat['chat'][a].split('★',2)[1])

df_chat=pd.DataFrame(data={'name':name,'chat_kor':chat_kor})

In [ ]:
import requests
# 파파고 스트링 번역

def papago_translate(context):
    try:
        USER_ID = "1rjN3zrT5BjNEDfGVaW9" # 개발자센터에서 발급받은 Client ID 값
        USER_SECRET = "7HmOEBInZY" # 개발자센터에서 발급받은 Client Secret 값
        url = "https://openapi.naver.com/v1/papago/n2mt"
        headers= {"X-Naver-Client-Id": USER_ID, "X-Naver-Client-Secret":USER_SECRET}
        params = {"source": "ko", "target": "en", "text": context}
        response = requests.post(url, headers=headers, data=params)
        res = response.json()
        return res['message']['result']['translatedText']
    except:
        return "PAPAGO_FAILED"

# talk_list 번역
def conv_translate(talk_list):
    translated_list = []
    for i in talk_list:
        new = papago_translate(i)
        translated_list.append(new)
    return translated_list

In [ ]:
df_chat['chat_eng']=conv_translate(df_chat['chat_kor'])

chat_for_viewing=conv_translate(df_chat['chat_kor'])

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer=SentimentIntensityAnalyzer()
def chat_compound(a):
    score=senti_analyzer.polarity_scores(a)['compound']
    return score
def chat_labeling(a):
    score=senti_analyzer.polarity_scores(a)['compound']
    if score>=0.3:
        return 'Good'
    elif score<=-0.3:
        return 'Bad'
    else:
        return 'SoSo'
    return score
df_chat['compound']=df_chat['chat_eng'].apply(lambda x:chat_compound(x))
df_chat['labeling']=df_chat['chat_eng'].apply(lambda x:chat_labeling(x))
        

df_chat['name']=df_chat['name'].str.replace(' ','')